ID3

In [8]:
data = [
    ['Sunny', 'Hot', 'High', 'Weak', 'No'],
    ['Sunny', 'Hot', 'High', 'Strong', 'No'],
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Weak', 'No'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Strong', 'No'],
]
features = ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [9]:
import math
from collections import Counter

In [10]:
import math
from collections import Counter

def entropy(data):
    labels = [row[-1] for row in data]
    counts = Counter(labels)
    total = len(labels)
    return -sum((count/total) * math.log2(count/total) for count in counts.values())

def info_gain(data, attr_index):
    total_entropy = entropy(data)
    attr_values = [row[attr_index] for row in data]
    unique_values = set(attr_values)
    subsets = [[row for row in data if row[attr_index] == value] for value in unique_values]
    weighted_entropy = sum((len(subset)/len(data)) * entropy(subset) for subset in subsets)
    return total_entropy - weighted_entropy

def id3(data, features):
    labels = [row[-1] for row in data]
    if labels.count(labels[0]) == len(labels):
        return labels[0]
    if not features:
        return Counter(labels).most_common(1)[0][0]

    gains = [info_gain(data, i) for i in range(len(features))]
    best = gains.index(max(gains))
    best_feature = features[best]

    tree = {best_feature: {}}
    unique_values = set(row[best] for row in data)
    for value in unique_values:
        subset = [row[:best] + row[best+1:] for row in data if row[best] == value]
        sub_features = features[:best] + features[best+1:]
        tree[best_feature][value] = id3(subset, sub_features)

    return tree


In [11]:
tree_id3 = id3(data, features)
print(tree_id3)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Overcast': 'Yes'}}


C45

In [12]:
def split_info(data, attr_index):
    attr_values = [row[attr_index] for row in data]
    counts = Counter(attr_values)
    total = len(attr_values)
    return -sum((count/total) * math.log2(count/total) for count in counts.values())

def gain_ratio(data, attr_index):
    ig = info_gain(data, attr_index)
    si = split_info(data, attr_index)
    return ig / si if si != 0 else 0

def c45(data, features):
    labels = [row[-1] for row in data]
    if labels.count(labels[0]) == len(labels):
        return labels[0]
    if not features:
        return Counter(labels).most_common(1)[0][0]

    ratios = [gain_ratio(data, i) for i in range(len(features))]
    best = ratios.index(max(ratios))
    best_feature = features[best]

    tree = {best_feature: {}}
    unique_values = set(row[best] for row in data)
    for value in unique_values:
        subset = [row[:best] + row[best+1:] for row in data if row[best] == value]
        sub_features = features[:best] + features[best+1:]
        tree[best_feature][value] = c45(subset, sub_features)

    return tree

In [13]:
tree_c45 = c45(data, features)
print(tree_c45)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Overcast': 'Yes'}}


CART

In [14]:
def gini(data):
    labels = [row[-1] for row in data]
    counts = Counter(labels)
    total = len(labels)
    return 1 - sum((count/total)**2 for count in counts.values())

def gini_index(data, attr_index):
    attr_values = [row[attr_index] for row in data]
    unique_values = set(attr_values)
    subsets = [[row for row in data if row[attr_index] == value] for value in unique_values]
    return sum((len(subset)/len(data)) * gini(subset) for subset in subsets)

def cart(data, features):
    labels = [row[-1] for row in data]
    if labels.count(labels[0]) == len(labels):
        return labels[0]
    if not features:
        return Counter(labels).most_common(1)[0][0]

    ginis = [gini_index(data, i) for i in range(len(features))]
    best = ginis.index(min(ginis))
    best_feature = features[best]

    tree = {best_feature: {}}
    unique_values = set(row[best] for row in data)
    for value in unique_values:
        subset = [row[:best] + row[best+1:] for row in data if row[best] == value]
        sub_features = features[:best] + features[best+1:]
        tree[best_feature][value] = cart(subset, sub_features)

    return tree

In [15]:
tree_cart = cart(data, features)
print(tree_cart)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Overcast': 'Yes'}}
